In [59]:
# #This  program is created by rakesh from crddmirror system on Tue 01 Dec 2020 11:22:29 PM IST.
#License: Attribution-NonCommercial-ShareAlike 4.0 International. For feedback, please email at rakesh4osdd@gmail.com
# check for updated code jupyter file name 'hpidb2.mitab_plus.txt' in MITAB 2.5 format
import csv
import re
import fileinput

In [60]:
# define input and output file with taxid mapping file
input1='hpidb2.mitab_plus.txt'
output1='hpi-'+input1
both2='both-'+input1
pathogen3='pathogen-'+input1
mapping_taxid='mapping-734_taxid_hpidb.tsv'

In [36]:
# set output file for writing
#fieldnames = [ 'SN', 'protein1', 'protein2', 'host', 'partner']
fieldnames = [ 'host_protein1', 'pathogen_protein2', 'host', 'pathogen', 'pathogen_category']
csvfile1=open(output1, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
output = csv.DictWriter(csvfile1,fieldnames=fieldnames, delimiter='\t')
output.writeheader()

csvfile2=open(both2, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
both = csv.DictWriter(csvfile2,fieldnames=fieldnames, delimiter='\t')
both.writeheader()

csvfile3=open(pathogen3, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
path = csv.DictWriter(csvfile3,fieldnames=fieldnames, delimiter='\t')
path.writeheader()
#path.writerow({'SN': '1', 'protein1': 'protein1', 'protein2': 'protein2' , 'host': 'host', 'partner': 'partner2'})

65

In [37]:
def update_taxid(path_taxid):
    with fileinput.input(mapping_taxid) as f:
        for line in f:
            taxid=line.split('\t')
            if re.search( rf'\b{taxid[0]}\b', path_taxid):
                return str(taxid[1][:-2])
#update_taxid('taxid:272624')

In [38]:
# code for filter MI-TAB 2.5 file | col16,17,10,11
with open(input1, encoding='utf-8', errors='ignore') as csvfile:
    hpi = csv.reader(csvfile, delimiter='\t')
    next(hpi, None)  # skip the headers
    for i, row in enumerate(hpi):
        if i >=0 :
            #print(row[16])
            #remove PRO and extract ':' saparated id
            row[15]=row[15].split('-PRO_')[0].split(':')[1]
            row[16]=row[16].split('-PRO_')[0].split(':')[1]
            protein1=row[15]
            protein2=row[16]
            # search for homo sapiens 9606
            pattern1=r'\b9606\b.*'
            # search for pair
            pattern2= r"\b9606\b[^:].*\b9606\b.*"
            #print(row[2]+row[3])
            string=row[9]+'\t'+row[10]
            host=row[9]
            pathogen=row[10]
            #print(row[0], row[1], row[9],update_taxid(row[10]))
            if re.search(pattern2, string):
                partner2=pathogen
                #both.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                both.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2, 'pathogen_category': row[18]})
            elif re.search(pattern1, host):
                # for homogenious hame of host
                host='taxid:9606(Homo sapiens)'
                partner2= update_taxid(pathogen)
                #output.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                output.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2, 'pathogen_category': row[18]}) 
            elif re.search(pattern1, pathogen):
                #print(row[0], row[1], row[9],row[10])
                protein1=row[16] # interchange of protein
                protein2=row[15]
                # for homogenious hame of host
                host='taxid:9606(Homo sapiens)'
                partner2=update_taxid(row[9])
                #output.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                output.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2, 'pathogen_category': row[18]})
            else:
                #print(row[0], row[1], row[9],row[10])
                partner2=pathogen
                path.writerow({'host_protein1': row[15] , 'pathogen_protein2': row[16] , 'host': row[9], 'pathogen': row[10], 'pathogen_category': row[18]})
                #print("No match",row)

csvfile1.flush()
csvfile2.flush()
csvfile3.flush()

In [39]:
# check output files and stats
! head -n2 {output1} {both2} {pathogen3}
! wc -l {output1} {both2} {pathogen3} ; wc -l {input1}
#! rm {output1} {both2} {pathogen3}
! tail -n +2 hpi-{input1} |cut -f1,2|sort -u|wc -l ; 
! tail -n +2 {input1} |cut -f1,2|sort -u|wc -l

==> hpi-hpidb2.mitab_plus.txt <==
host_protein1	pathogen_protein2	host	pathogen	pathogen_category
P62820	Q5ZWZ3	taxid:9606(Homo sapiens)	taxid:272624(Legionella pneumophila subsp. pneumophila str. Philadelphia 1)	BACTERIA

==> both-hpidb2.mitab_plus.txt <==
host_protein1	pathogen_protein2	host	pathogen	pathogen_category

==> pathogen-hpidb2.mitab_plus.txt <==
host_protein1	pathogen_protein2	host	pathogen	pathogen_category
P22518	Q5ZRQ0	taxid:10090(mouse|Mus musculus)	taxid:272624(legph|Legionella pneumophila subsp. pneumophila)	BACTERIA
  66033 hpi-hpidb2.mitab_plus.txt
      1 both-hpidb2.mitab_plus.txt
   3756 pathogen-hpidb2.mitab_plus.txt
  69790 total
69788 hpidb2.mitab_plus.txt
50256
54879


In [40]:
csvfile1.close()
csvfile2.close()
csvfile3.close()

In [41]:
# check for Mt Protein exist in the hpi-{input1} file using Mt_Proteins_7432_working_uniprot_ids.txt and mt-protein-uniprot.txt
! head -n1 hpi-{input1} > mnDNA_hpi-{input1}
! grep -a -w -f ../Mt_Proteins_7432_working_uniprot_ids.txt hpi-{input1} >> mnDNA_hpi-{input1}
! wc -l mnDNA_hpi-{input1}; wc -l hpi-{input1}
! head -n2 mnDNA_hpi-{input1} hpi-{input1}

28647 mnDNA_hpi-hpidb2.mitab_plus.txt
66033 hpi-hpidb2.mitab_plus.txt
==> mnDNA_hpi-hpidb2.mitab_plus.txt <==
host_protein1	pathogen_protein2	host	pathogen	pathogen_category
P62820	Q5ZWZ3	taxid:9606(Homo sapiens)	taxid:272624(Legionella pneumophila subsp. pneumophila str. Philadelphia 1)	BACTERIA

==> hpi-hpidb2.mitab_plus.txt <==
host_protein1	pathogen_protein2	host	pathogen	pathogen_category
P62820	Q5ZWZ3	taxid:9606(Homo sapiens)	taxid:272624(Legionella pneumophila subsp. pneumophila str. Philadelphia 1)	BACTERIA


In [42]:
# summay count uniq values in each column of the mt protein HPI file
c1 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f1|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f1
c2 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f2|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f2
c3 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f3|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f3
c4 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f4|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f4
c12 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f1,2|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f1,2
c_all = ! tail -n +2 mnDNA_hpi-{input1}|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1}
print (c1,c2,c3,c4,c12, c_all)

['2733', 'host_protein1'] ['3707', 'pathogen_protein2'] ['1', 'host'] ['410', 'pathogen'] ['22325', 'host_protein1\tpathogen_protein2'] ['22669', 'host_protein1\tpathogen_protein2\thost\tpathogen\tpathogen_category']


In [43]:
# split file into mDNA and nDNA list
! grep -a -v -w -f ../mt-protein-uniprot.txt mnDNA_hpi-{input1} > nDNA_hpi-{input1}
! head -n1 mnDNA_hpi-{input1} > mDNA_hpi-{input1}
! grep -a -w -f ../mt-protein-uniprot.txt mnDNA_hpi-{input1} >> mDNA_hpi-{input1}
! wc -l mDNA_hpi-{input1} nDNA_hpi-{input1} ; wc -l mnDNA_hpi-{input1}
! head -n3 mDNA_hpi-{input1} nDNA_hpi-{input1} mnDNA_hpi-{input1}

    110 mDNA_hpi-hpidb2.mitab_plus.txt
  28538 nDNA_hpi-hpidb2.mitab_plus.txt
  28648 total
28647 mnDNA_hpi-hpidb2.mitab_plus.txt
==> mDNA_hpi-hpidb2.mitab_plus.txt <==
host_protein1	pathogen_protein2	host	pathogen	pathogen_category
P00403	P03431	taxid:9606(Homo sapiens)	taxid:211044(Influenza A virus (A/Puerto Rico/8/1934(H1N1)))	VIRUS
P00403	P03428	taxid:9606(Homo sapiens)	taxid:211044(Influenza A virus (A/Puerto Rico/8/1934(H1N1)))	VIRUS

==> nDNA_hpi-hpidb2.mitab_plus.txt <==
host_protein1	pathogen_protein2	host	pathogen	pathogen_category
P62820	Q5ZWZ3	taxid:9606(Homo sapiens)	taxid:272624(Legionella pneumophila subsp. pneumophila str. Philadelphia 1)	BACTERIA
P60953	A0A0H2ZTM2	taxid:9606(Homo sapiens)	taxid:243243(Mycobacterium avium 104)	BACTERIA

==> mnDNA_hpi-hpidb2.mitab_plus.txt <==
host_protein1	pathogen_protein2	host	pathogen	pathogen_category
P62820	Q5ZWZ3	taxid:9606(Homo sapiens)	taxid:272624(Legionella pneumophila subsp. pneumophila str. Philadelphia 1)	BACTERIA
P60953	A

In [44]:
# to create mapping file
#! tail -n +2 mnDNA_hpi-{input1}|cut -f4|sort -u > 557_pathogen_taxid.txt

In [45]:
!wc -l 557_pathogen_taxid.txt
! head -n3 mnDNA_hpi-{input1}

557 557_pathogen_taxid.txt
host_protein1	pathogen_protein2	host	pathogen	pathogen_category
P62820	Q5ZWZ3	taxid:9606(Homo sapiens)	taxid:272624(Legionella pneumophila subsp. pneumophila str. Philadelphia 1)	BACTERIA
P60953	A0A0H2ZTM2	taxid:9606(Homo sapiens)	taxid:243243(Mycobacterium avium 104)	BACTERIA


In [46]:
# to create mapping file
#! cat 557_pathogen_taxid.txt 557_pathogen_taxid_ncbi.txt |sort > mapping_557_pathogen_taxid.txt

In [47]:
! head -n2 mnDNA_hpi-{input1}

In [48]:
#split file into virus and bacteria and mDNA, nDNA with stats
"""! head -n1 mnDNA_hpi-{input1} | cut -f1-4 > mnDNA_hpi-bacteria-{input1}
! grep -w 'BACTERIA' mnDNA_hpi-{input1} | cut -f1-4 >> mnDNA_hpi-bacteria-{input1}
! head -n1 mnDNA_hpi-{input1} | cut -f1-4 > mnDNA_hpi-virus-{input1}
! grep -w 'VIRUS' mnDNA_hpi-{input1} | cut -f1-4 >> mnDNA_hpi-virus-{input1}
! wc -l  mnDNA_hpi-bacteria-{input1} mnDNA_hpi-virus-{input1} ; wc -l mnDNA_hpi-{input1}
! head -n3 mnDNA_hpi-{input1}  mnDNA_hpi-virus-{input1} mnDNA_hpi-{input1}"""

pathogen_type=['BACTERIA', 'VIRUS']
for pt in pathogen_type:
    ! head -n1 mnDNA_hpi-{input1} | cut -f1-4 > mnDNA_hpi-{pt}-{input1}
    ! grep -w {pt} mnDNA_hpi-{input1} | cut -f1-4 >> mnDNA_hpi-{pt}-{input1}
    ! head -n1 mnDNA_hpi-{pt}-{input1} > mDNA_hpi-{pt}-{input1}
    ! grep -a -w -f ../mt-protein-uniprot.txt mnDNA_hpi-{pt}-{input1} >> mDNA_hpi-{pt}-{input1}
    ! grep -a -v -w -f ../mt-protein-uniprot.txt mnDNA_hpi-{pt}-{input1} > nDNA_hpi-{pt}-{input1}
    ! wc -l mDNA_hpi-{pt}-{input1} nDNA_hpi-{pt}-{input1}
    mnDNA= ! ls mDNA_hpi-{pt}-{input1} nDNA_hpi-{pt}-{input1}
    for db_list in mnDNA:
        ! wc -l {db_list}
        for i in range(4):
            header = ! head -n1 {db_list} |cut -f{i+1}
            count = ! tail -n +2 {db_list} |cut -f{i+1}  |sort -u|wc -l
            print (header[0], count[0]) 

     1 mDNA_hpi-BACTERIA-hpidb2.mitab_plus.txt
  3180 nDNA_hpi-BACTERIA-hpidb2.mitab_plus.txt
  3181 total
1 mDNA_hpi-BACTERIA-hpidb2.mitab_plus.txt
host_protein1 0
pathogen_protein2 0
host 0
pathogen 0
3180 nDNA_hpi-BACTERIA-hpidb2.mitab_plus.txt
host_protein1 1088
pathogen_protein2 1630
host 1
pathogen 55
    110 mDNA_hpi-VIRUS-hpidb2.mitab_plus.txt
  22930 nDNA_hpi-VIRUS-hpidb2.mitab_plus.txt
  23040 total
110 mDNA_hpi-VIRUS-hpidb2.mitab_plus.txt
host_protein1 13
pathogen_protein2 50
host 1
pathogen 33
22930 nDNA_hpi-VIRUS-hpidb2.mitab_plus.txt
host_protein1 2364
pathogen_protein2 1015
host 1
pathogen 340


In [49]:
! grep -v 'BACTERIA\|VIRUS' mnDNA_hpi-{input1}|cut -f5|sort -u

In [51]:
! sort -u mnDNA_hpi-VIRUS-hpidb2.mitab_plus.txt|uniq |wc -l

18210


In [53]:
! du -sh hpi-hpidb2.mitab_plus.txt

5.8M	hpi-hpidb2.mitab_plus.txt


In [54]:
! head -n3 hpi-hpidb2.mitab_plus.txt

In [57]:
! tail -n +2 hpi-hpidb2.mitab_plus.txt|cut -f1|sort -u|wc -l
! tail -n +2 hpi-hpidb2.mitab_plus.txt|cut -f2|sort -u|wc -l
! tail -n +2 hpi-hpidb2.mitab_plus.txt|cut -f3|sort -u|wc -l
! tail -n +2 hpi-hpidb2.mitab_plus.txt|cut -f4|sort -u|wc -l

9579
5829
1
564


In [58]:
column4file='hpi-hpidb2.mitab_plus.txt'
! wc -l {column4file}
for i in range(4):
    header = ! head -n1 {column4file} |cut -f{i+1}
    count = ! tail -n +2 {column4file} |cut -f{i+1}  |sort -u|wc -l
    print (header[0], count[0])

66033 hpi-hpidb2.mitab_plus.txt
host_protein1 9579
pathogen_protein2 5829
host 1
pathogen 564


In [63]:
!  grep -a -w -f ../Mt_Proteins_7432_working_uniprot_ids.txt hpi-{input1}|cut -f1|sort -u|grep '-'

O00410-3
O15294-3
O15350-2
O15350-3
O15350-4
O43852-3
O95197-3
O95292-2
P02545-2
P02545-3
P06748-3
P07900-2
P07910-2
P08631-2
P20290-2
P20807-4
P21333-2
P29590-2
P29590-3
P29590-4
P29590-5
P29590-8
P35613-2
P36956-4
P37231-2
P41235-2
P46379-2
P48960-2
P54252-1
P63010-2
Q01082-3
Q01105-2
Q04323-2
Q05086-2
Q05086-3
Q13554-3
Q14141-4
Q14596-2
Q15019-3
Q16630-2
Q16630-3
Q6RW13-2
Q7Z6G3-2
Q8NFF5-2
Q96CV9-2
Q96ED9-2
Q96EY1-3
Q96KN7-4
Q9H0B3-3
Q9HCM9-2
Q9NQC3-3
Q9NZL4-3
Q9UH92-3
Q9UM00-1
Q9Y2Z0-2
Q9Y5P4-2
Q9Y6N9-4


In [64]:
! grep '-' ../Mt_Proteins_7432_working_uniprot_ids.txt

In [65]:
! pwd

/home/rakesh/mitolink_Nov2020/pathogen-host-interaction/remapped_7457_HPI/HPIDB
